패션 이미지 분류 문제에서 로지스틱 회귀를 쓰는 것보다 인공 신경망 모델을 쓰는 게 더 성능이 좋았다. 다른 방법을 사용해서 성능을 더 향상시켜보자.

## 1. 데이터 준비
#### <7-1. 인공 신경망> 에서 했던대로 데이터를 준비하기


In [1]:
from tensorflow import keras

# 1. 데이터를 불러오고
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

# 2. 픽셀값 0~255 범위를 0~1범위로 변환하고
train_scaled = train_input / 255.0

# 3. 28* 28 크기의 3차원 배열을 가진 60,000개 샘플을 2차원 배열로 바꿔주고
train_scaled = train_scaled.reshape(-1, 28*28)

# 4. train_test_split() 함수 사용해서 검증 세트를 떼어놓고
from sklearn.model_selection import train_test_split
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size = 0.2, random_state = 42)

# 5. 데이터 크기 확인
print(train_scaled.shape, train_target.shape)
print(val_scaled.shape, val_target.shape)


2023-06-08 15:43:48.308547: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(48000, 784) (48000,)
(12000, 784) (12000,)


## 2. 심층 신경망 모델 만들기
- 심층 신경망(Deep Neural Network, DNN)은 입력층(input layer)과 출력층(output layer) 사이에 여러 개의 은닉층(hidden layer)들로 이뤄진 인공신경망(Artificial Neural Network, ANN)이다.

### 2-1. 인공 신경망 모델에 층을 2개 추가하는 방법
- 입력층과 출력층 사이에 은닉층을 추가한다.

##### 활성화 함수
- 밀집층을 추가할 때 `activation` 매개변수에 활성화 함수를 지정해준다.
- 분류를 위한 확률 출력의 경우, 활성화 함수를 사용하는데, '은닉층'이라는 밀집층을 추가할 때는 활성화 함수를 비교적 자유롭게 선택할 수 있다.
    - 대표적 활성화 함수: 시그모이드, 렐루 등...
- 회귀를 위한 확률 출력의 경우, 회귀는 임의의 숫자이므로 활성화 함수를 적용할 필요가 없다. Dense 층의 `activation` 매개변수에 값을 지정하지 않고 출력층의 선형 방정식 계산을 그대로 출력한다.    


In [2]:
dense1 = keras.layers.Dense(100, activation = 'sigmoid', input_shape = (784,))
dense2 = keras.layers.Dense(10, activation = 'softmax')

- `model = keras.Sequential([hidden layer, output layer])`: 은닉층 - 출력층 순으로 작성해준다. 

In [3]:
model = keras.Sequential([dense1, dense2])
# 2개의 밀집층을 가진 신경망 모델을 만들기 위해 케라스의 Sequential() 클래스 이용
# Sequential() 클래스의 객체에 밀집층의 객체인 [dense1, dense2]를 전달
# `name` 매개변수로 모델의 이름을 지정할 수 있다.

2023-06-08 15:43:51.727807: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.summary()
# 모델 정보 확인하기

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


### 2-2. 인공 신경망 모델에 층을 추가하는 방법

In [5]:
# not running: only for practice

model = keras.Sequential([
    keras.layers.Dense(100, activation = 'sigmoid', input_shape = (784,), name = 'hidden')
    , keras.layers.Dense(10, activation = 'softmax', name = 'output')
], name = '패션 MNIST 모델')

model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


### 2-3. 인공 신경망 모델에 층을 추가하는 방법

In [6]:
# only for practice

model = keras.Sequential()
model.add(keras.layers.Dense(100, activation = 'sigmoid', input_shape = (784,)))
model.add(keras.layers.Dense(10, activation = 'softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


## 3. 모델 훈련
- 케라스는 모델 훈련 전 설정 단계에서 손실함수를 꼭 지정해주고 훈련을 시켜야 한다.

In [7]:
model.compile(loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 2s 878us/step - loss: 0.5597 - accuracy: 0.8074
Epoch 2/5
1500/1500 [==============================] - 1s 861us/step - loss: 0.4066 - accuracy: 0.8535
Epoch 3/5
1500/1500 [==============================] - 1s 867us/step - loss: 0.3717 - accuracy: 0.8654
Epoch 4/5
1500/1500 [==============================] - 1s 869us/step - loss: 0.3501 - accuracy: 0.8741
Epoch 5/5
1500/1500 [==============================] - 1s 857us/step - loss: 0.3325 - accuracy: 0.8802


In [8]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 0s 587us/step - loss: 0.3474 - accuracy: 0.8733


[0.3473699688911438, 0.8733333349227905]

- **이전에 만든 인공 신경망에 밀집층을 2개를 사용해서 모델 훈련을 했을 때 정확도가 약간은 높아진 것을 확인할 수 있다. (인공 신경망 검증 세트 정확도 점수 0.850)**